# <center> Pokemon-goDB

<img src='mongo.jpeg'>

Start by running `mongod` in your terminal.

Import libraries.

In [ ]:
import sqlite3 
import pandas as pd
import requests
import json
import pymongo

Connect to Mongo, create a database and a collection.

In [ ]:
client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
poke_db = client['pokemon_db']
poke_collection = poke_db['pokemon_collection']

Create an entry and insert it into the collection.

In [ ]:
pikachu = {'name': 'Pikachu', 'primary_type': 'Electric', 'secondary_type': 'None', 'level':25}
results = poke_collection.insert_one(pikachu)

Run a query to find all items in the collection.

In [ ]:
q = poke_collection.find({})
for i in q:
    print(i)

Insert multiple entries at once.

In [ ]:
charmander = {'name': 'Charmander', 'primary_type': 'Fire', 'secondary_type': 'None', 'level':12}
bulbasaur = {'name': 'Bulbasaur',  'primary_type': 'Grass', 'secondary_type': 'Poison', 'level':10}
squirtle = {'name': 'Squirtle',  'primary_type': 'Water', 'secondary_type': 'None', 'level':8}

starters_list = [charmander, bulbasaur, squirtle]

results = poke_collection.insert_many(starters_list)

Query just the names of all the entered Pokemon without the IDs.

In [ ]:
q = poke_collection.find({}, {'name':1, '_id':0})
for i in q:
    print(i)

Query the Pokemon with a level less than 20.

https://docs.mongodb.com/manual/reference/operator/query/#query-selectors

In [ ]:
q = poke_collection.find({'level':{'$lt': 20}}, {'name':1, '_id':0})
for i in q:
    print(i)

Delete an entry.

In [ ]:
d = poke_collection.delete_one({'name': 'Pikachu'})
print(d.deleted_count)

In [ ]:
q = poke_collection.find({}, {'name':1, '_id':0})
for i in q:
    print(i)

Delete all entries without a secondary type.

In [ ]:
d = poke_collection.delete_many({'secondary_type':'None'})
print(d.deleted_count)

In [ ]:
q = poke_collection.find({}, {'name':1, '_id':0})
for i in q:
    print(i)

Delete the entire collection.

In [ ]:
d = poke_collection.delete_many({})

In [ ]:
q = poke_collection.find({}, {'name':1, '_id':0})
for i in q:
    print(i)

## ETL

### Extract data from API

Documentation - https://pokeapi.co/docs/v2.html

Create base URL.

In [ ]:
base_url = 'https://pokeapi.co/api/v2'

Create URL to get info on Pikachu. Make request to URL and check response keys.

In [ ]:
pika_url = base_url + '/pokemon/pikachu'
response = requests.get(pika_url)
pika_json = response.json()
pika_json.keys()

Get the name, primary type, and secondary type.

In [ ]:
name = pika_json['name']

types = pika_json['types']

type1 = types[0]['type']['name']
type1_url = types[0]['type']['url']

type2 = None if len(types)<2 else types[1]['type']['name']
type2_url = None if len(types)<2 else types[1]['type']['url']

print(name, type1, type1_url, type2, type2_url)

Now we want to find the types Pikachu is weak to.

Using the '/type' endpoint, find info on the electric type.

In [ ]:
url = 'https://pokeapi.co/api/v2/type/electric'
response = requests.get(url).json()
response.keys()

Check the 'damage_relations' dictionary.

In [ ]:
response['damage_relations'].keys()

Grab the types supereffective against electric from the 'double_damage_from' key. Iterate through to get the weaknesses.

In [ ]:
response['damage_relations']['double_damage_from']

In [ ]:
weaknesses = [i['name'] for i in response['damage_relations']['double_damage_from']]
weaknesses

# <center> Activity

<img src='pokedex.png'>

Professor Oak has tasked you with compiling info on Pokemon. Using the PokeAPI, extract data on all 151 original Pokemon and load that data into a Mongo database. <br>
<br>
You should include the following info for each Pokemon:
    - Name
    - Height
    - Weight
    - Primary type
    - Secondary Type
    - Weaknesses
    - Moves

<b>Note</b>: For weaknesses, make sure you look at both the primary type and secondary type. The list should only contain each weaknesses once even if both the primary and secondary types share a weaknesses (maybe use a Python set).

<b>Hint</b>: The first 151 Pokemon have IDs from 1-151. You can use that to collect info on each one in a loop.

# <center> Bonus

### Mongo Queries

If you finish early, try to solve these questions using queries. Here are some links that might be helpful:
- https://docs.mongodb.com/manual/reference/operator/query/#query-selectors
- https://api.mongodb.com/python/current/api/pymongo/cursor.html#pymongo.cursor.Cursor.sort
- https://api.mongodb.com/python/current/examples/aggregation.html

What are the top 5 tallest Pokemon?

What are the top 5 lightest Pokemon?

What Pokemon are weak to Ghost type attacks?

What Pokemon has the most number of weaknesses?